In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

# Define the URL for the SEC EDGAR search page
url = "https://www.sec.gov/files/company_tickers_exchange.json"

headers = {
   "User-Agent": "your.email@email.com"
}
# Fetch the HTML content of the search page
response = requests.get(url,headers=headers)
html_content = response.content

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

json_url = f"https://data.sec.gov/api/xbrl/companyconcept/CIK0001652044/us-gaap/AccountsPayableCurrent.json"
json_response = requests.get(json_url,headers=headers)
json_content = json_response.json()
print(json_content)
print(soup.prettify())

In [ ]:


# Parse the JSON content from the BeautifulSoup object
json_data = json.loads(soup.text)

# Convert the JSON data into a pandas DataFrame
df_cik_lookup = pd.DataFrame(json_data['data'], columns=json_data['fields'])


# df_cik_lookup['cik']=df_cik_lookup['cik'].astype(str).zfill(10)
df_cik_lookup

In [ ]:
json_data_list = []

# Define headers for the request
headers = {
    'User-Agent': 'Your Name (your_email@example.com)',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'data.sec.gov'
}

# Loop through the first 100 CIKs
for i in range(100):
    cik = df_cik_lookup['cik'][i]
    cik=str(cik).zfill(10)
    print(cik)
    json_url = f"https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/AccountsPayableCurrent.json"
    json_response = requests.get(json_url, headers=headers)
    
    # Check if the request was successful
    if json_response.status_code == 200:
        json_content = json_response.json()
        json_data_list.append(json_content)
    else:
        print(f"Failed to fetch data for CIK: {cik}")

# Convert the list of JSON content into a DataFrame
df_json_content = pd.DataFrame(json_data_list)
df_json_content.to_csv('data.csv', index=False)


In [ ]:
df_json_content

In [5]:
df_units=pd.json_normalize(df_json_content['units'])

In [ ]:
df_units

In [ ]:
# Define a function to extract the 'val' field from the JSON data
def extract_val(json_data):
    normalized_data = pd.json_normalize(json_data)
    filtered_data = normalized_data[normalized_data['form'] == '10-K']
    return filtered_data['val'].tolist()

def extract_filled(json_data):
    normalized_data = pd.json_normalize(json_data)
    filtered_data = normalized_data[normalized_data['form'] == '10-K']
    return filtered_data['end'].tolist()

# Apply the function to the 'USD' column and create a new column 'value'
df_json_content['value'] = df_units['USD'].apply(lambda x: extract_val(x) if isinstance(x, list) else [])
df_json_content['dates_filed'] = df_units['USD'].apply(lambda x: extract_filled(x) if isinstance(x, list) else [])

# Display the updated DataFrame
df_json_content


In [ ]:

import matplotlib.pyplot as plt

# Plot graphs of value for the first 5 companies
for i in range(5):
    company_name = df_json_content['entityName'][i]
    values = df_json_content['value'][i]
    time = df_json_content['dates_filed'][i]
    
    plt.figure(figsize=(10, 5))
    plt.plot(time, values, marker='o')
    plt.title(f'Value Over Time for {company_name}')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
    plt.ylim(min(values) * 0.9, max(values) * 1.1)  # Set y-axis limits with some padding
    plt.grid(True)
    plt.show()

In [ ]:
json_data_list = []

# Define headers for the request
headers = {
    'User-Agent': 'Your Name (your_email@example.com)',
    'Accept-Encoding': 'gzip, deflate',
    'Host': 'data.sec.gov'
}

# Loop through the first 100 CIKs
for i in range(100):
    cik = df_cik_lookup['cik'][i]
    cik=str(cik).zfill(10)
    print(cik)
    json_url = f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json"
    json_response = requests.get(json_url, headers=headers)
    
    # Check if the request was successful
    if json_response.status_code == 200:
        json_content = json_response.json()
        json_data_list.append(json_content)
    else:
        print(f"Failed to fetch data for CIK: {cik}")

# Convert the list of JSON content into a DataFrame
df_json_companyfacts = pd.DataFrame(json_data_list)
df_json_companyfacts.to_csv('data2.csv', index=False)

In [ ]:
df_json_companyfacts

In [19]:
df_facts=pd.json_normalize(df_json_companyfacts['facts'])

In [ ]:
df_facts

In [ ]:
df_facts.columns

In [ ]:
url="https://data.sec.gov/submissions/CIK0000822663.json"
# Fetch the JSON content from the URL
response = requests.get(url,headers=headers)
json_content = response.json()

# Convert JSON content to a string
json_str = json.dumps(json_content)

# Parse the string using BeautifulSoup
soup = BeautifulSoup(json_str, 'html.parser')

# Print the parsed content
print(soup.prettify())


In [ ]:
print(json_content.keys())

In [ ]:
print(json_content['fiscalYearEnd'])

In [ ]:
import mongodb as mdb

mdb.get_mongodb_client()



In [ ]:
mdb.download_cik_ticker_map()

In [ ]:
tiker = "GOOGL"
cik = mdb.cik_from_ticker(tiker)
cik

In [ ]:
mdb.download_submissions_documents(cik,"10-K",5)

In [4]:
mdb.download_all_cik_submissions(cik)

In [5]:
mdb.download_submissions_documents(cik,("10-k"),5)

In [2]:
import pandas as pd

In [3]:
df=pd.read_excel("transposed_df_v2 (3).xlsx")

In [ ]:
df

In [4]:
df['cik'] = df['cik'].apply(lambda x: mdb.add_trailing_to_cik(x))

In [6]:
cik=df['cik'].iloc[0]

In [ ]:
cik

In [ ]:
for cik in df['cik']:
    mdb.download_submissions_documents(cik,("10-K"),5)

In [ ]:
mdb.download_submissions_documents()

In [ ]:
cik

In [ ]:
filing_url = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
doc = mdb.get_collection("submissions").find({"_id":filing_url }).next()
doc

In [18]:
import requests

url='https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
headers={
    "User-Agent": "your.email@email.com"
}
response= requests.get(url,headers)
r = response.text

In [ ]:
import mongodb as mdb
import datetime

url='https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
mdb.download_document(url,cik,'10-K',None,None)

In [3]:
r=response.text

In [ ]:
r

In [ ]:
response

In [ ]:
import mongodb as mdb

mdb.get_mongodb_client()

In [ ]:
filing_url = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
doc = mdb.get_collection("documents").find({"_id":filing_url }).next()
doc

In [ ]:
r=mdb.make_edgar_request(filing_url)

In [ ]:
r.text

In [1]:
import mongodb as mdb
import requests
filing_url='https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
headers={
    "User-Agent": "radnom@ten.edu",
    "Accept-Encoding": "gzip, deflate"
    }
r=requests.get(filing_url,headers=headers)

In [ ]:
import mongodb as mdb
filing_url='https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'

r=mdb.make_edgar_request(filing_url)

In [ ]:
r.text

In [ ]:
import mongodb as mdb
ticker="GOOGL"
cik = mdb.cik_from_ticker(ticker)
cik

In [3]:
filing_url='https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'
mdb.download_document(filing_url,cik,'10-K',None,None)

In [ ]:
import mongodb as mdb

mdb.get_mongodb_client()

In [1]:
import pandas as pd
import mongodb as mdb

df=pd.read_excel("transposed_df_v2 (3).xlsx")
df['cik'] = df['cik'].apply(lambda x: mdb.add_trailing_to_cik(x))

In [ ]:
for cik in df['cik'][:50]:
    mdb.download_submissions_documents(cik, ("10-K"), 5)

In [ ]:
import mongodb as mdb

for doc in mdb.get_collection("documents").find():
    mdb.parse_document(doc)

In [ ]:
import analyzer as an
for parsed_doc in mdb.get_collection("parsed_documents").find():
    an.sections_summary(parsed_doc)

In [1]:
import mongodb as mdb
filing_url = 'https://www.sec.gov/Archives/edgar/data/1652044/000165204423000016/goog-20221231.htm'

doc = mdb.get_collection("documents").find({"_id":filing_url}).next()

In [ ]:
mdb.parse_document(doc)

In [3]:
parsed_doc = mdb.get_collection("parsed_documents").find({"_id":filing_url}).next()

In [4]:
import analyzer as an

restructured_doc=an.restructure_parsed_10k(parsed_doc)

In [ ]:
section_text = restructured_doc["business"]["text"]
section_text

In [ ]:
len(section_text)

In [ ]:
chain_type = "refine"
model = "gpt-4o-mini"
verbose = True

# get summary from openAI model
print(f"business original_len: {len(section_text)} use {model} w/ chain {chain_type}")
summary, cost = an.summarize_section(section_text, model, chain_type, verbose)

In [ ]:
print(f"BULLET POINTS")
for el in summary:
    print(el)
print(f"cost: {cost} in USD")

In [ ]:
an.sections_summary(parsed_doc)

In [ ]:
import mongodb as mdb
for cik in df['cik'][:50]:
    mdb.download_submissions_documents(cik, ("10-K"), 5)